In [ ]:
# Problem 4
# 4.1
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
import math
from sklearn.linear_model import SGDClassifier
import pylab as pl
import random
import pdb
import sklearn as sk
from cvxopt import matrix, solvers

In [346]:
def svmgauss(X, Y, C, gamma, tol):
    Xt = X.T
    Yt = Y.T
    # Data range
    a = min(X[0])-0.5
    b = max(X[0])+0.5
    c = min(X[1])-0.5
    d = max(X[1])+0.5
    
    # matrices for solver
    A = matrix(Yt.T, tc = 'd')
    b_mat = matrix(0, tc = 'd')
    G = matrix(np.vstack(((-1*np.identity(len(Yt))), np.identity(len(Yt)))))
    h = matrix(np.hstack((np.zeros(len(Yt)), np.repeat(C, len(Yt))))[np.newaxis].T)
    q = matrix(np.repeat(-1, len(Yt))[np.newaxis].T, tc = 'd')
    pairwise_sq_dists = squareform(pdist(Xt, 'sqeuclidean'))
    K = Yt*Y*scipy.exp(-gamma*pairwise_sq_dists)
    P = matrix(K, tc = 'd')
    
    # Call solver
    sol = solvers.qp(P,q,G,h,A,b_mat)
    alpha = np.asarray((sol['x']))
    status = sol['status']
    # Threshold alpha
    tol = tol
    alpha[alpha < tol] = 0
    theta = np.dot(scipy.exp(-gamma*pairwise_sq_dists), np.multiply(alpha, Yt))
    cond = (alpha > tol).reshape(-1)
    cond2 = ((alpha < C) > tol).reshape(-1)
    Xt_sv = Xt[cond]
    sq_dists = (cdist(Xt_sv, Xt_sv, 'sqeuclidean'))
    Z = np.dot(scipy.exp(-gamma*sq_dists), np.multiply(alpha[cond], Yt[cond]))    
    SV = Xt[cond] #support vectors
    bias_vec = Yt[cond]-Z    
    test=np.histogram(bias_vec, bins=50000)
    max(test[0])
    bias = np.mean(test[1][test[0]==max(test[0])]) #most frequent bias
    bias
    return cond, C, gamma, SV, alpha, bias, a, b, c, d, bias_vec

In [393]:
##### 4.3

import scipy
from random import shuffle


from scipy.spatial.distance import pdist, squareform, cdist

import time

def shuffleData(n, X, Y):
    indices = range(n)
    shuffle(indices)
    X = [X[i] for i in indices]
    Y = [Y[i] for i in indices]
    return (X,Y)

def train(X,Y,n, features, lmbda, max_epochs):
    t = 0
    epoch = 0
    w = np.zeros((features, 1))
    b = np.zeros((1,1))
    for epoch in xrange(max_epochs):
        for i in xrange(n):
            t += 1
            eta = float(1)/(t*lmbda)
            if np.dot(Y[i],np.dot(np.transpose(w), X[i]) + b) < 1:
                w = np.add((1-eta*lmbda) * w,  eta * Y[i] * np.transpose(X[i][np.newaxis]))
                b = np.add(b, (eta*Y[i]))
            else:
                w = (1-eta*lmbda) * w

        X, Y = shuffleData(n, X, Y)
    return (w,b)

def computeKernel(X, gamma, n):
    K = np.zeros((n,n));
    for i in xrange(n):
        for j in xrange(n):
            K[i][j] = math.exp(- gamma * np.dot(np.transpose(X[i]-X[j]), X[i]-X[j]))
    return K


def train_gaussianSVM(X, Y, n, K, lmbda, max_epochs):
    t = 0
    epoch = 0
    alpha = np.zeros((n,1)) 
    for epoch in xrange(max_epochs):
        for i in xrange(n):
            t += 1
            eta = float(1)/(t*lmbda)
            if np.dot(Y[i], np.dot(np.transpose(alpha),K[:,i])) < 1:
                alpha[i] = (1-eta*lmbda)*alpha[i] + eta*Y[i]
            else:
                alpha[i] = (1-eta*lmbda)*alpha[i]
    return alpha


def train(X,Y,n, features, lmbda, max_epochs):
    t = 0
    epoch = 0
    w = np.zeros((features, 1))
    b = np.zeros((1,1))
    for epoch in xrange(max_epochs):
        for i in xrange(n):
            t += 1
            eta = float(1)/(t*lmbda)
            if np.dot(Y[i],np.dot(np.transpose(w), X[i]) + b) < 1:
                w = np.add((1-eta*lmbda) * w,  eta * Y[i] * np.transpose(X[i][np.newaxis]))
                b = np.add(b, (eta*Y[i]))
            else:
                w = (1-eta*lmbda) * w

        X, Y = shuffleData(n, X, Y)
    return (w,b)


def calcTime(X, Y):
	X = X.T
	Y = Y[np.newaxis]
	print X.shape
	print Y.shape
	tol = 1e-4
	gamma = 1
	svmlin_times = []
	svmgauss_times = []
	Cs= [1e-5, 1e-2, 10, 1e4, 1e10, 1e15]
	for i in xrange(len(Cs)):
		C = Cs[i]
		start_time = time.time()
		svmlin(X,Y,C, tol)
		time_elapsed = time.time() - start_time
		svmlin_times.append(time_elapsed)
		start_time_gauss = time.time()
		svmgauss(X,Y,C, gamma, tol)
		time_elapsed_gauss = time.time() - start_time_gauss
		svmgauss_times.append(time_elapsed_gauss)


	return svmlin_times, svmgauss_times

def calcTimePegasos(X,Y, epochs):

	n_train = X.shape[0]
	features_train = X.shape[1]
	gamma = 1
	svmlin_times = []
	svmgauss_times = []
	Cs= [1e-5, 1e-2, 10, 1e4, 1e10, 1e15]
	lmbdas = [1 / x for x in Cs]
        for i in xrange(len(Cs)):
            lmbda = lmbdas[i]
            start_time = time.time()
            w,b = train(X, Y, n_train, features_train, lmbda, epochs)
            time_elapsed = time.time() - start_time
            svmlin_times.append(time_elapsed)
            start_time_gauss = time.time()
            K = computeKernel(X, gamma, n_train)
            alpha = train_gaussianSVM(X, Y, n_train, K, lmbda, epochs)
            time_elapsed_gauss = time.time() - start_time_gauss
            svmgauss_times.append(time_elapsed_gauss)

        return svmlin_times, svmgauss_times




max_epochs = 100
time_elapsed, time_elapsed_gauss = calcTime(Xtrain1_7, Ytrain1_7)
time_elapsed_peg, time_elapsed_gauss_peg = calcTimePegasos(Xtrain1_7, Ytrain1_7, max_epochs)

print('LIN SVM TIMES')
for time in time_elapsed: print time

print('GAUSS SVM TIMES')
for time in time_elapsed_gauss: print time

print('PEGASOS LIN SVM TIMES')
for time in time_elapsed_peg: print time

print('PEGASOS GAUSS SVM TIMES')
for time in time_elapsed_gauss_peg: print time


(784, 400)
(1, 400)
     pcost       dcost       gap    pres   dres
 0: -2.6124e-03 -6.4829e-03  8e+02  3e+01  6e-12
 1: -2.5899e-03 -6.2414e-03  9e+00  3e-01  4e-12
 2: -1.0211e-03 -4.4782e-03  3e-01  9e-03  5e-12
 3: -2.2269e-04 -3.9258e-03  2e-02  5e-04  9e-13
 4: -1.0208e-04 -2.0298e-03  4e-03  7e-05  2e-13
 5: -5.3402e-05 -1.0050e-03  2e-03  3e-05  7e-14
 6: -1.7373e-05 -5.0635e-04  7e-04  1e-05  3e-14
 7: -1.0726e-05 -9.9728e-05  1e-04  2e-06  1e-14
 8: -8.4522e-06 -2.0448e-05  1e-05  1e-07  7e-15
 9: -9.5261e-06 -1.4312e-05  6e-06  4e-08  5e-15
10: -1.0208e-05 -1.1550e-05  1e-06  4e-20  6e-15
11: -1.0639e-05 -1.0823e-05  2e-07  4e-20  6e-15
12: -1.0711e-05 -1.0720e-05  9e-09  4e-20  6e-15
Optimal solution found.


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


     pcost       dcost       gap    pres   dres
 0: -1.0576e+02 -2.2605e+01  1e+03  4e+01  4e-16
 1: -1.9359e+01 -6.6897e-01  8e+01  2e+00  3e-16
 2: -2.0679e-01 -7.8080e-03  9e-01  3e-02  5e-16
 3: -4.9570e-03 -7.6991e-03  1e-02  2e-04  5e-16
 4: -3.6575e-03 -5.0654e-03  1e-03  2e-17  4e-16
 5: -3.9797e-03 -4.0111e-03  3e-05  1e-17  4e-16
 6: -3.9998e-03 -4.0001e-03  3e-07  9e-18  9e-17
 7: -4.0000e-03 -4.0000e-03  3e-09  2e-17  2e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -2.6135e-03 -4.0093e+00  8e+02  3e+01  7e-12
 1: -2.5992e-03 -3.9313e+00  1e+01  3e-01  5e-12
 2: -1.7679e-03 -1.3405e+00  2e+00  2e-02  5e-12
 3: -3.3352e-04 -1.8061e-01  2e-01  2e-03  3e-12
 4: -1.4847e-04 -2.3215e-02  3e-02  3e-04  6e-13
 5: -5.0300e-05 -4.3055e-03  5e-03  4e-05  1e-13
 6: -2.2654e-05 -1.1990e-03  1e-03  1e-05  4e-14
 7: -1.1648e-05 -2.2463e-04  3e-04  2e-06  1e-14
 8: -8.3187e-06 -4.3644e-05  4e-05  2e-07  7e-15


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:37: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


 9: -9.4332e-06 -2.0045e-05  1e-05  6e-08  6e-15
10: -9.5643e-06 -1.2920e-05  3e-06  3e-17  6e-15
11: -1.0419e-05 -1.1215e-05  8e-07  4e-17  6e-15
12: -1.0659e-05 -1.0794e-05  1e-07  4e-17  7e-15
13: -1.0713e-05 -1.0717e-05  3e-09  4e-17  6e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0: -1.0659e+02 -2.9665e+01  1e+03  4e+01  1e-16
 1: -2.1635e+01 -8.2761e+00  8e+01  2e+00  3e-16
 2: -3.5356e+00 -7.2306e+00  4e+00  2e-14  7e-16
 3: -3.8682e+00 -4.0180e+00  1e-01  1e-14  2e-16
 4: -3.9758e+00 -3.9773e+00  2e-03  1e-14  3e-16
 5: -3.9769e+00 -3.9769e+00  2e-05  6e-15  1e-16
 6: -3.9769e+00 -3.9769e+00  2e-07  1e-14  3e-16
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.4359e-01 -4.0740e+03  9e+03  1e-01  7e-11
 1:  1.8594e-01 -1.2349e+02  2e+02  3e-03  5e-11
 2:  3.0883e-02 -6.8676e+00  1e+01  2e-04  4e-11
 3:  5.6412e-03 -5.1779e-01  9e-01  1e-05  6e-12
 4:  1.2605e-03 -6.2107e-02  1e-01  1e-06  9e-13
 5:  4.0611e-04 -1.0873

33:  5.3501e+00 -5.8891e+00  1e+01  1e-16  8e-12
34:  7.6562e-01 -8.4495e-01  2e+00  4e-17  3e-12
35:  1.0932e-01 -1.2148e-01  2e-01  7e-17  1e-12
36:  1.5513e-02 -1.7558e-02  3e-02  1e-16  4e-13
37:  2.1634e-03 -2.5745e-03  5e-03  2e-16  1e-13
38:  2.8523e-04 -3.9248e-04  7e-04  2e-16  5e-14
39:  2.8817e-05 -6.6931e-05  1e-04  1e-16  2e-14
40: -2.2367e-06 -2.0356e-05  2e-05  1e-16  9e-15
41: -7.2777e-06 -1.3152e-05  6e-06  2e-16  6e-15
42: -9.2334e-06 -1.2409e-05  3e-06  2e-16  5e-15
43: -1.0399e-05 -1.1132e-05  7e-07  2e-16  6e-15
44: -1.0653e-05 -1.0798e-05  1e-07  7e-17  6e-15
45: -1.0712e-05 -1.0718e-05  6e-09  1e-16  6e-15
Optimal solution found.
     pcost       dcost       gap    pres   dres
 0:  2.2598e+31 -7.0355e+31  9e+31  2e+01  9e-02
 1:  4.9275e+30 -5.5510e+30  1e+31  2e+02  7e-02
 2:  7.1755e+29 -8.4558e+29  2e+30  8e+01  3e-02
 3:  1.0348e+29 -1.1493e+29  2e+29  7e+01  1e-02
 4:  1.4858e+28 -1.6426e+28  3e+28  1e+01  5e-03
 5:  2.1305e+27 -2.3461e+27  4e+27  7e+00  2e-

For the questions below, you will first need to construct training, validation and test sets. You will do thatby collecting images for some subset of the digits, that will be labeled as +1, and some other subset that willbe labeled as -1.  Take the first 200 points for each digit and use them for training, use the next 150 pointsfor validation and the final 150 points as a test set.  If you are classifying (2,4) vs (3,5), then your trainingset will have 800 samples, your validation and training sets will have 600 samples each. Note that you willconstruct a vector of ones of length 400 and a vector of -1s, also of length 400, and these will be the labelsfor your training set, and so on.Be careful with the sizes of these data sets, you probably don’t wantto use all the samples in the data files.


After constructing these data sets, you should also construct corresponding sets that are “normalized”, sothat each input feature (pixel) is in the range [-1, 1]. Because each feature in this dataset has minimum value0 and maximum value 255, you can do this normalization by transforming the wholeXmatrix (element-wise) to2X/255−1.Try (at least) the following pairwise classification tasks: 1 vs 7, 3 vs 5, 4 vs 9, (0, 2, 4, 6, 8) vs (1, 3, 5,7, 9).

1.  Show the classification accuracy of your logistic regression from problem 1 and the linear SVM classifierfrom problem 2 on the MNIST data.  Compare and contrast the performance of each classifier (bothtraining  and  testing).   Does  normalization  of  the  data  matter?   Look  at  some  of  the  images  of  mis-classified digits, do they make sense?

In [2]:
def getData(digit, ind1, ind2):
    data = pl.loadtxt('/Users/becca/GitHub/6.867/code_hw2/data/mnist_digit_' + str(digit) + '.csv')
    # Returns column matrices
    X = data[ind1:ind2]
    return X

In [ ]:
Xtrain1 = getData(1, 0, 200)
Xtrain7 = getData(7, 0, 200)
Xval1 = getData(1, 200, 350)
Xval7 = getData(7, 200, 350)
Xtest1 = getData(1, 350, 500)
Xtest7 = getData(7, 350, 500)

Xtrain3 = getData(3, 0, 200)
Xtrain5 = getData(5, 0, 200)
Xval3 = getData(3, 200, 350)
Xval5 = getData(5, 200, 350)
Xtest3 = getData(3, 350, 500)
Xtest5 = getData(5, 350, 500)

Xtrain4 = getData(4, 0, 200)
Xtrain9 = getData(9, 0, 200)
Xval4 = getData(4, 200, 350)
Xval9 = getData(9, 200, 350)
Xtest4 = getData(4, 350, 500)
Xtest9 = getData(9, 350, 500)

In [ ]:
def svmlin(X, Y, C, tol):
    Xt = X.T
    Yt = Y.T
    # Data range
    a = min(X[0])-0.5
    b = max(X[0])+0.5
    c = min(X[1])-0.5
    d = max(X[1])+0.5
    
    # matrices for solver
    A = matrix(Yt.T, tc = 'd')
    b_mat = matrix(0, tc = 'd')
    G = matrix(np.vstack(((-1*np.identity(len(Yt))), np.identity(len(Yt)))))
    h = matrix(np.hstack((np.zeros(len(Yt)), np.repeat(C, len(Yt))))[np.newaxis].T)
    q = matrix(np.repeat(-1, len(Yt))[np.newaxis].T, tc = 'd')
    K = Yt*Xt
    P = matrix(np.dot(K, K.T), tc = 'd')
    # Call solver
    sol = solvers.qp(P,q,G,h,A,b_mat)
    alpha = np.asarray((sol['x']))
    status = sol['status']
    # Threshold alpha
    tol = tol
    alpha[alpha < tol] = 0
    theta = np.dot(X, np.multiply(alpha, Yt))
    
    cond = (alpha > tol).reshape(-1)
    SV = Xt[cond] #support vectors
    bias_vec = Yt[cond] - np.dot(Xt[cond], theta)
    test=np.histogram(bias_vec, bins=50000)
    max(test[0])
    bias = np.mean(test[1][test[0]==max(test[0])]) #most frequent bias
    bias
    return C, SV, theta, bias, a, b, c, d, bias_vec

In [341]:
svmlin(Xtrain1_7.T, Ytrain1_7[np.newaxis], 1, tol = 1e-4)


     pcost       dcost       gap    pres   dres
 0: -2.7328e-04 -4.0135e+02  2e+03  1e+00  1e-11
 1:  3.8312e-03 -1.0575e+02  1e+02  2e-02  1e-11
 2: -2.3430e-04 -6.0994e+00  7e+00  1e-03  1e-11
 3: -1.1061e-04 -5.0113e-01  6e-01  8e-05  3e-12
 4: -4.7118e-05 -6.5966e-02  7e-02  1e-05  7e-13
 5: -5.2031e-06 -1.1844e-02  1e-02  2e-06  1e-13
 6:  3.9680e-06 -2.7970e-03  3e-03  3e-07  4e-14
 7:  1.0807e-06 -3.5293e-04  4e-04  4e-08  1e-14
 8: -5.8270e-06 -3.1233e-05  3e-05  2e-09  7e-15
 9: -8.2535e-06 -1.8264e-05  1e-05  7e-10  6e-15
10: -9.5825e-06 -1.3204e-05  4e-06  2e-16  7e-15
11: -1.0406e-05 -1.1288e-05  9e-07  2e-16  6e-15
12: -1.0647e-05 -1.0812e-05  2e-07  2e-16  7e-15
13: -1.0712e-05 -1.0718e-05  6e-09  2e-16  6e-15
Optimal solution found.


/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:32: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 50001 but corresponding boolean dimension is 50000


(1, 400)

In [ ]:
Xtrain02468 = np.reshape(np.array([getData(0, 0, 200), getData(2, 0, 200), getData(4, 0, 200), 
                        getData(6, 0, 200), getData(8, 0, 200)]), (1000, 784))
Xtrain13579 = np.reshape(np.array([Xtrain1, Xtrain3, Xtrain5, Xtrain7, Xtrain9]), (1000, 784))

Xval02468 = np.reshape(np.array([getData(0, 200, 350), getData(2, 200, 350), getData(4, 200, 350), 
                        getData(6, 200, 350), getData(8, 200, 350)]), (750, 784))
Xval13579 = np.reshape(np.array([Xval1, Xval3, Xval5, Xval7, Xval9]), (750, 784))

In [63]:
Xtest02468 = np.reshape(np.array([getData(0, 350, 500), getData(2, 350, 500), getData(4, 350, 500), 
                        getData(6, 350, 500), getData(8, 350, 500)]), (750, 784))
Xtest13579 = np.reshape(np.array([Xtest1, Xtest3, Xtest5, Xtest7, Xtest9]), (750, 784))

In [ ]:
def normalize(train, val, test):
    trainnorm = np.zeros((train.shape[0], train.shape[1]))
    valnorm = np.zeros((val.shape[0], val.shape[1]))
    testnorm = np.zeros((test.shape[0], test.shape[1]))

    for i in xrange(train.shape[0]):
        trainnorm[i] = 2*train[i]/255-1
    for i in xrange(val.shape[0]):
        valnorm[i] = 2*val[i]/255-1
    for i in xrange(test.shape[0]):
        testnorm[i] = 2*test[i]/255-1
        
    return trainnorm, valnorm, testnorm

In [ ]:
Xtrain1norm, Xval1norm, Xtest1norm = normalize(Xtrain1, Xval1, Xtest1)
Xtrain7norm, Xval7norm, Xtest7norm = normalize(Xtrain7, Xval7, Xtest7)

Xtrain3norm, Xval3norm, Xtest3norm = normalize(Xtrain3, Xval3, Xtest3)
Xtrain5norm, Xval5norm, Xtest5norm = normalize(Xtrain5, Xval5, Xtest5)

Xtrain4norm, Xval4norm, Xtest4norm = normalize(Xtrain4, Xval4, Xtest4)
Xtrain9norm, Xval9norm, Xtest9norm = normalize(Xtrain9, Xval9, Xtest9)

Xtrain02468norm, Xval02468norm, Xtest02468norm = normalize(Xtrain02468, Xval02468, Xtest02468)
Xtrain13579norm, Xval13579norm, Xtest13579norm = normalize(Xtrain13579, Xval13579, Xtest13579)

In [71]:
Xtrain1_7 = np.concatenate((Xtrain1, Xtrain7), axis = 0)
Xtrain1_7norm = np.concatenate((Xtrain1norm, Xtrain7norm), axis = 0)

Xval1_7 = np.concatenate((Xval1, Xval7), axis = 0)
Xval1_7norm = np.concatenate((Xval1norm, Xval7norm), axis = 0)

Xtest1_7 = np.concatenate((Xtest1, Xtest7), axis = 0)
Xtest1_7norm = np.concatenate((Xtest1norm, Xtest7norm), axis = 0)

Ytrain1_7 = np.concatenate((np.repeat(-1, len(Xtrain1)), np.repeat(1, len(Xtrain7))), axis = 0)
Yval1_7 = np.concatenate((np.repeat(-1, len(Xval1)), np.repeat(1, len(Xval7))), axis = 0)
Ytest1_7 = np.concatenate((np.repeat(-1, len(Xtest1)), np.repeat(1, len(Xtest7))), axis = 0)


Xtrain3_5 = np.concatenate((Xtrain3, Xtrain5), axis = 0)
Xtrain3_5norm = np.concatenate((Xtrain3norm, Xtrain5norm), axis = 0)

Xval3_5 = np.concatenate((Xval3, Xval5), axis = 0)
Xval3_5norm = np.concatenate((Xval3norm, Xval5norm), axis = 0)

Xtest3_5 = np.concatenate((Xtest3, Xtest5), axis = 0)
Xtest3_5norm = np.concatenate((Xtest3norm, Xtest5norm), axis = 0)

Ytrain3_5 = np.concatenate((np.repeat(-1, len(Xtrain3)), np.repeat(1, len(Xtrain5))), axis = 0)
Yval3_5 = np.concatenate((np.repeat(-1, len(Xval3)), np.repeat(1, len(Xval5))), axis = 0)
Ytest3_5 = np.concatenate((np.repeat(-1, len(Xtest3)), np.repeat(1, len(Xtest5))), axis = 0)


Xtrain4_9 = np.concatenate((Xtrain4, Xtrain9), axis = 0)
Xtrain4_9norm = np.concatenate((Xtrain4norm, Xtrain9norm), axis = 0)

Xval4_9 = np.concatenate((Xval4, Xval9), axis = 0)
Xval4_9norm = np.concatenate((Xval4norm, Xval9norm), axis = 0)

Xtest4_9 = np.concatenate((Xtest4, Xtest9), axis = 0)
Xtest4_9norm = np.concatenate((Xtest4norm, Xtest9norm), axis = 0)

Ytrain4_9 = np.concatenate((np.repeat(-1, len(Xtrain4)), np.repeat(1, len(Xtrain9))), axis = 0)
Yval4_9 = np.concatenate((np.repeat(-1, len(Xval4)), np.repeat(1, len(Xval9))), axis = 0)
Ytest4_9 = np.concatenate((np.repeat(-1, len(Xtest4)), np.repeat(1, len(Xtest9))), axis = 0)


Xtrainmany = np.concatenate((Xtrain02468, Xtrain13579), axis = 0)
Xtrainmanynorm = np.concatenate((Xtrain02468norm, Xtrain13579norm), axis = 0)

Xvalmany = np.concatenate((Xval02468, Xval13579), axis = 0)
Xvalmanynorm = np.concatenate((Xval02468norm, Xval13579norm), axis = 0)

Xtestmany = np.concatenate((Xtest02468, Xtest13579), axis = 0)
Xtestmanynorm = np.concatenate((Xtest02468norm, Xtest13579norm), axis = 0)

Ytrainmany = np.concatenate((np.repeat(-1, len(Xtrain02468)), np.repeat(1, len(Xtrain13579))), axis = 0)
Yvalmany = np.concatenate((np.repeat(-1, len(Xval02468)), np.repeat(1, len(Xval13579))), axis = 0)
Ytestmany = np.concatenate((np.repeat(-1, len(Xtest02468)), np.repeat(1, len(Xtest13579))), axis = 0)


In [ ]:

### done constructing datasets..

In [146]:
from __future__ import division
x17valaccl2 = []
x17testaccl2 = []
x17valaccl2norm = []
x17testaccl2norm = []
for lamda in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    clfl2 = sk.linear_model.LogisticRegression(penalty = 'l2', tol = 1e-10, C = 1/lamda, 
                                              fit_intercept = True, random_state = 10, 
                                         max_iter = 1000, intercept_scaling = 1e6)

    clfl2.fit(Xtrain1_7, Ytrain1_7)
    # Percent Agreement
    #sum(clfl2.predict(Xtrain1_7) == Ytrain1_7)/len(Ytrain1_7)*100.0000 #100% for lamda 1000
    x17valaccl2.append(sum(clfl2.predict(Xval1_7) == Yval1_7)/len(Yval1_7)*100.0000) #99% for lamda 1000
    x17testaccl2.append(sum(clfl2.predict(Xtest1_7) == Ytest1_7)/len(Yval1_7)*100.0000) #98.67% for lamda 1000
    
    clfl2.fit(Xtrain1_7norm, Ytrain1_7)
    x17valaccl2norm.append(sum(clfl2.predict(Xval1_7norm) == Yval1_7)/len(Yval1_7)*100.0000) 
    x17testaccl2norm.append(sum(clfl2.predict(Xtest1_7norm) == Ytest1_7)/len(Yval1_7)*100.0000) 
    

x35valaccl2 = []
x35testaccl2 = []
x35valaccl2norm = []
x35testaccl2norm = []
for lamda in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    clfl2 = sk.linear_model.LogisticRegression(penalty = 'l2', tol = 1e-10, C = 1/lamda, 
                                              fit_intercept = True, random_state = 10, 
                                         max_iter = 1000, intercept_scaling = 1e6)

    clfl2.fit(Xtrain3_5, Ytrain3_5)
    # Percent Agreement
    #sum(clfl2.predict(Xtrain1_7) == Ytrain1_7)/len(Ytrain1_7)*100.0000 
    x35valaccl2.append(sum(clfl2.predict(Xval3_5) == Yval3_5)/len(Yval3_5)*100.0000) 
    x35testaccl2.append(sum(clfl2.predict(Xtest3_5) == Ytest3_5)/len(Yval3_5)*100.0000) 
    
    clfl2.fit(Xtrain3_5norm, Ytrain3_5)
    x35valaccl2norm.append(sum(clfl2.predict(Xval3_5norm) == Yval3_5)/len(Yval3_5)*100.0000) 
    x35testaccl2norm.append(sum(clfl2.predict(Xtest3_5norm) == Ytest3_5)/len(Yval3_5)*100.0000) 
    
    
x49valaccl2 = []
x49testaccl2 = []
x49valaccl2norm = []
x49testaccl2norm = []
for lamda in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    clfl2 = sk.linear_model.LogisticRegression(penalty = 'l2', tol = 1e-10, C = 1/lamda, 
                                              fit_intercept = True, random_state = 10, 
                                         max_iter = 1000, intercept_scaling = 1e6)

    clfl2.fit(Xtrain4_9, Ytrain4_9)
    # Percent Agreement
    #sum(clfl2.predict(Xtrain1_7) == Ytrain1_7)/len(Ytrain1_7)*100.0000 
    x49valaccl2.append(sum(clfl2.predict(Xval4_9) == Yval4_9)/len(Yval4_9)*100.0000) 
    x49testaccl2.append(sum(clfl2.predict(Xtest4_9) == Ytest4_9)/len(Yval4_9)*100.0000) 
    
    clfl2.fit(Xtrain4_9norm, Ytrain4_9)
    x49valaccl2norm.append(sum(clfl2.predict(Xval4_9norm) == Yval4_9)/len(Yval4_9)*100.0000) 
    x49testaccl2norm.append(sum(clfl2.predict(Xtest4_9norm) == Ytest4_9)/len(Yval4_9)*100.0000) 
    
xmanyvalaccl2 = []
xmanytestaccl2 = []
xmanyvalaccl2norm = []
xmanytestaccl2norm = []
for lamda in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    clfl2 = sk.linear_model.LogisticRegression(penalty = 'l2', tol = 1e-10, C = 1/lamda, 
                                              fit_intercept = True, random_state = 10, 
                                         max_iter = 1000, intercept_scaling = 1e6)

    clfl2.fit(Xtrainmany, Ytrainmany)
    # Percent Agreement
    #sum(clfl2.predict(Xtrain1_7) == Ytrain1_7)/len(Ytrain1_7)*100.0000 
    xmanyvalaccl2.append(sum(clfl2.predict(Xvalmany) == Yvalmany)/len(Yvalmany)*100.0000) 
    xmanytestaccl2.append(sum(clfl2.predict(Xtestmany) == Ytestmany)/len(Yvalmany)*100.0000) 
    
    clfl2.fit(Xtrainmanynorm, Ytrainmany)
    xmanyvalaccl2norm.append(sum(clfl2.predict(Xvalmanynorm) == Yvalmany)/len(Yvalmany)*100.0000) 
    xmanytestaccl2norm.append(sum(clfl2.predict(Xtestmanynorm) == Ytestmany)/len(Yvalmany)*100.0000) 

In [144]:
from sklearn import svm

xmanyvalaccsvm = []
xmanytestaccsvm = []
xmanyvalaccsvmnorm = []
xmanytestaccsvmnorm = []
for C in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    model = svm.SVC(kernel='linear', C = C) 
    model.fit(Xtrainmany, np.ndarray.flatten(Ytrainmany))
    xmanyvalaccsvm.append(sum(model.predict(Xvalmany) == Yvalmany)/len(Yvalmany)*100)
    xmanytestaccsvm.append(sum(model.predict(Xtestmany) == Ytestmany)/len(Yvalmany)*100)
    
    model.fit(Xtrainmanynorm, np.ndarray.flatten(Ytrainmany))
    xmanyvalaccsvmnorm.append(sum(model.predict(Xvalmanynorm) == Yvalmany)/len(Yvalmany)*100)
    xmanytestaccsvmnorm.append(sum(model.predict(Xtestmanynorm) == Ytestmany)/len(Yvalmany)*100)

In [152]:
x17valaccsvm = []
x17testaccsvm = []
x17valaccsvmnorm = []
x17testaccsvmnorm = []
for C in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    model = svm.SVC(kernel='linear', C = C) 
    model.fit(Xtrain1_7, np.ndarray.flatten(Ytrain1_7))
    x17valaccsvm.append(sum(model.predict(Xval1_7) == Yval1_7)/len(Yval1_7)*100)
    x17testaccsvm.append(sum(model.predict(Xtest1_7) == Ytest1_7)/len(Yval1_7)*100)

    model.fit(Xtrain1_7norm, np.ndarray.flatten(Ytrain1_7))
    x17valaccsvmnorm.append(sum(model.predict(Xval1_7norm) == Yval1_7)/len(Yval1_7)*100)
    x17testaccsvmnorm.append(sum(model.predict(Xtest1_7norm) == Ytest1_7)/len(Yval1_7)*100)

In [ ]:
x35valaccsvm = []
x35testaccsvm = []
x35valaccsvmnorm = []
x35testaccsvmnorm = []
for C in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    model = svm.SVC(kernel='linear', C = C) 
    model.fit(Xtrain3_5, np.ndarray.flatten(Ytrain3_5))
    x35valaccsvm.append(sum(model.predict(Xval3_5) == Yval3_5)/len(Yval3_5)*100)
    x35testaccsvm.append(sum(model.predict(Xtest3_5) == Ytest3_5)/len(Yval3_5)*100)

    model.fit(Xtrain3_5norm, np.ndarray.flatten(Ytrain3_5))
    x35valaccsvmnorm.append(sum(model.predict(Xval3_5norm) == Yval3_5)/len(Yval3_5)*100)
    x35testaccsvmnorm.append(sum(model.predict(Xtest3_5norm) == Ytest3_5)/len(Yval3_5)*100)
    
x49valaccsvm = []
x49testaccsvm = []
x49valaccsvmnorm = []
x49testaccsvmnorm = []
for C in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    model = svm.SVC(kernel='linear', C = C) 
    model.fit(Xtrain4_9, np.ndarray.flatten(Ytrain4_9))
    x49valaccsvm.append(sum(model.predict(Xval4_9) == Yval4_9)/len(Yval4_9)*100)
    x49testaccsvm.append(sum(model.predict(Xtest4_9) == Ytest4_9)/len(Yval4_9)*100)

    model.fit(Xtrain4_9norm, np.ndarray.flatten(Ytrain4_9))
    x49valaccsvmnorm.append(sum(model.predict(Xval4_9norm) == Yval4_9)/len(Yval4_9)*100)
    x49testaccsvmnorm.append(sum(model.predict(Xtest4_9norm) == Ytest4_9)/len(Yval4_9)*100)
    

In [269]:
# Gauss kernel
from sklearn import svm

xmanyvalaccsvmg = []
xmanytestaccsvmg = []
xmanyvalaccsvmnormg = []
xmanytestaccsvmnormg = []
for C in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    for gamma in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
        model = svm.SVC(kernel='rbf', C = C, gamma = gamma) 
        model.fit(Xtrainmany, np.ndarray.flatten(Ytrainmany))
        xmanyvalaccsvmg.append(sum(model.predict(Xvalmany) == Yvalmany)/len(Yvalmany)*100)
        xmanytestaccsvmg.append(sum(model.predict(Xtestmany) == Ytestmany)/len(Yvalmany)*100)

        model.fit(Xtrainmanynorm, np.ndarray.flatten(Ytrainmany))
        xmanyvalaccsvmnormg.append(sum(model.predict(Xvalmanynorm) == Yvalmany)/len(Yvalmany)*100)
        xmanytestaccsvmnormg.append(sum(model.predict(Xtestmanynorm) == Ytestmany)/len(Yvalmany)*100)
        print gamma

1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30


In [271]:
# more gauss kernel
x17valaccsvmg = []
x17testaccsvmg = []
x17valaccsvmnormg = []
x17testaccsvmnormg = []
for C in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    for gamma in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
        model = svm.SVC(kernel='rbf', C = C, gamma = gamma) 
        model.fit(Xtrain1_7, np.ndarray.flatten(Ytrain1_7))
        x17valaccsvmg.append(sum(model.predict(Xval1_7) == Yval1_7)/len(Yval1_7)*100)
        x17testaccsvmg.append(sum(model.predict(Xtest1_7) == Ytest1_7)/len(Yval1_7)*100)

        model.fit(Xtrain1_7norm, np.ndarray.flatten(Ytrain1_7))
        x17valaccsvmnormg.append(sum(model.predict(Xval1_7norm) == Yval1_7)/len(Yval1_7)*100)
        x17testaccsvmnormg.append(sum(model.predict(Xtest1_7norm) == Ytest1_7)/len(Yval1_7)*100)
        print gamma


x35valaccsvmg = []
x35testaccsvmg = []
x35valaccsvmnormg = []
x35testaccsvmnormg = []
for C in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    for gamma in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
        model = svm.SVC(kernel='rbf', C = C, gamma = gamma) 
        model.fit(Xtrain3_5, np.ndarray.flatten(Ytrain3_5))
        x35valaccsvmg.append(sum(model.predict(Xval3_5) == Yval3_5)/len(Yval3_5)*100)
        x35testaccsvmg.append(sum(model.predict(Xtest3_5) == Ytest3_5)/len(Yval3_5)*100)

        model.fit(Xtrain3_5norm, np.ndarray.flatten(Ytrain3_5))
        x35valaccsvmnormg.append(sum(model.predict(Xval3_5norm) == Yval3_5)/len(Yval3_5)*100)
        x35testaccsvmnormg.append(sum(model.predict(Xtest3_5norm) == Ytest3_5)/len(Yval3_5)*100)
        print gamma
    
x49valaccsvmg = []
x49testaccsvmg = []
x49valaccsvmnormg = []
x49testaccsvmnormg = []
for C in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
    for gamma in [1e-5, 1e-2, 10, 1e4, 1e10, 1e30]:
        model = svm.SVC(kernel='rbf', C = C, gamma = gamma) 
        model.fit(Xtrain4_9, np.ndarray.flatten(Ytrain4_9))
        x49valaccsvmg.append(sum(model.predict(Xval4_9) == Yval4_9)/len(Yval4_9)*100)
        x49testaccsvmg.append(sum(model.predict(Xtest4_9) == Ytest4_9)/len(Yval4_9)*100)

        model.fit(Xtrain4_9norm, np.ndarray.flatten(Ytrain4_9))
        x49valaccsvmnormg.append(sum(model.predict(Xval4_9norm) == Yval4_9)/len(Yval4_9)*100)
        x49testaccsvmnormg.append(sum(model.predict(Xtest4_9norm) == Ytest4_9)/len(Yval4_9)*100)

        print gamma

1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30
1e-05
0.01
10
10000.0
10000000000.0
1e+30


In [299]:
# pick test acc corr to max val accuracy (if tie, picked min C)

svmg = np.array([x17testaccsvmg[30], 
               x17testaccsvmnormg[31], 
               x35testaccsvmg[6],
               x35testaccsvmnormg[31],
               x49testaccsvmg[6],
               x49testaccsvmnormg[31],
               xmanytestaccsvmg[6],
               xmanytestaccsvmnormg[31]])


In [303]:
# pick test acc corr to max val accuracy (if tie, picked min C, max lambda)
l2 = np.array([x17testaccl2[3], 
               x17testaccl2norm[2], 
               x35testaccl2[3],
               x35testaccl2norm[1],
               x49testaccl2[x49valaccl2.index(max(x49valaccl2))],
               x49testaccl2norm[x49valaccl2norm.index(max(x49valaccl2norm))],
               xmanytestaccl2[xmanyvalaccl2.index(max(xmanyvalaccl2))],
               xmanytestaccl2norm[xmanyvalaccl2norm.index(max(xmanyvalaccl2norm))]
              ])

svmpd = np.array([x17testaccsvm[0], 
               x17testaccsvmnorm[1], 
               x35testaccsvm[0],
               x35testaccsvmnorm[x35valaccsvmnorm.index(max(x35valaccsvmnorm))],
               x49testaccsvm[x49valaccsvm.index(max(x49valaccsvm))],
               x49testaccsvmnorm[x49valaccsvmnorm.index(max(x49valaccsvmnorm))],
               xmanytestaccsvm[xmanyvalaccsvm.index(max(xmanyvalaccsvm))],
               xmanytestaccsvmnorm[xmanyvalaccsvmnorm.index(max(xmanyvalaccsvmnorm))]
              ])



<module 'sklearn.svm' from '/anaconda/lib/python2.7/site-packages/sklearn/svm/__init__.pyc'>

In [304]:
import pandas as pd
import seaborn as sns
l2svmacc = pd.DataFrame(data = np.array([l2, svmpd]).T, columns = ['L2', 'Linear SVM'])
l2svmacc.index = ['1 vs 7', '1 vs 7 norm', '3 vs 5', '3 vs 5 norm', '4 vs 9', '4 vs 9 norm', '02468 vs 13579', '02468 vs 13579 norm']

svmacc =  pd.DataFrame(data = np.array([svmpd, svmg]).T, columns = ['Linear SVM', 'Gaussian RBF'])
svmacc.index = ['1 vs 7', '1 vs 7 norm', '3 vs 5', '3 vs 5 norm', '4 vs 9', '4 vs 9 norm', '02468 vs 13579', '02468 vs 13579 norm']

Optimum RBF gamma C:
17 : gamma = 1e-5, C = 1e30
17norm: gamma = 1e-2, C = 1e30
35: gamma = 1e-5, C = 1e-2
35 norm: gamma = 1e-2, C = 1e30
49: gamma = 1e-5, C = 1e-2
49 norm: gamma = 1e-2, C = 1e30
many norm: gamma = 1e-5, C = 1e-2
many norm: gamma = 1e-2, C = 1e30

In [323]:
C2 = np.repeat([1e-5, 1e-2, 10, 1e4, 1e10, 1e30], 6).T
gamma2 = np.tile([1e-5, 1e-2, 10, 1e4, 1e10, 1e30], 6).T
xmanytestaccsvmnormg
pd.DataFrame([C2, gamma2])

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,33,34,35
0,0.00001,0.00001,0.00001,0.00001,1.000000e-05,1.000000e-05,0.01000,0.01,0.01,0.01,...,1.000000e+10,1.000000e+10,1.000000e+10,1.000000e+10,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30,1.000000e+30
1,0.00001,0.01000,10.00000,10000.00000,1.000000e+10,1.000000e+30,0.00001,0.01,10.00,10000.00,...,1.000000e+01,1.000000e+04,1.000000e+10,1.000000e+30,1.000000e-05,1.000000e-02,1.000000e+01,1.000000e+04,1.000000e+10,1.000000e+30


In [ ]:
# pick test acc corr to max val accuracy (if tie, picked min C)

svmg = np.array([x17testaccsvmg[30], 
               x17testaccsvmnormg[31], 
               x35testaccsvmg[6],
               x35testaccsvmnormg[31],
               x49testaccsvmg[6],
               x49testaccsvmnormg[31],
               xmanytestaccsvmg[6],
               xmanytestaccsvmnormg[31]])

In [305]:
svmacc

,Linear SVM,Gaussian RBF
1 vs 7,98.666667,82.666667
1 vs 7 norm,98.666667,99.333333
3 vs 5,94.666667,65.333333
3 vs 5 norm,95.333333,99.000000
4 vs 9,94.333333,88.333333
4 vs 9 norm,93.666667,98.000000
02468 vs 13579,89.133333,76.200000
02468 vs 13579 norm,88.666667,97.400000


In [ ]:
# Plot SVM and L2 regression max test accuracy results
import seaborn as sns

sns.set(font_scale=1.6)
f,(ax1) = plt.subplots(1,1,sharey=True, figsize = (4,5))
g1 = sns.heatmap(l2svmacc, cmap = inferno, ax = ax1, annot = True, annot_kws={"size": 16})
plt.yticks(rotation=0) 
#ax1.set_title('Test Accuracy ')

plt.xlabel('')

plt.savefig('L2 vs linear SVM test acc.jpg', dpi= 300, bbox_inches='tight')

In [307]:
# Plot SVM and SVM gauss max test accuracy results
import seaborn as sns

sns.set(font_scale=1.6)
f,(ax1) = plt.subplots(1,1,sharey=True, figsize = (5,5))
g1 = sns.heatmap(svmacc, cmap = inferno, ax = ax1, annot = True, annot_kws={"size": 16})
plt.yticks(rotation=0) 
#ax1.set_title('Test Accuracy ')

plt.xlabel('')

plt.savefig('linear SVM vs gauss rbf test acc.jpg', dpi= 300, bbox_inches='tight')

In [ ]:
# Plot SVM and L2 regression max test accuracy results for Emily
import seaborn as sns

sns.set(font_scale=1.6)
f,(ax1) = plt.subplots(1,1,sharey=True, figsize = (4,5))
g1 = sns.heatmap(l2svmacc, cmap = viridis, ax = ax1, annot = True, annot_kws={"size": 16})
plt.yticks(rotation=0) 
#ax1.set_title('Test Accuracy ')

plt.xlabel('')

plt.savefig('L2 vs linear SVM test acc for Emily.jpg', dpi= 300, bbox_inches='tight')

In [308]:
# Plot SVM and SVM gauss max test accuracy results
import seaborn as sns

sns.set(font_scale=1.6)
f,(ax1) = plt.subplots(1,1,sharey=True, figsize = (5,5))
g1 = sns.heatmap(svmacc, cmap = viridis, ax = ax1, annot = True, annot_kws={"size": 16})
plt.yticks(rotation=0) 
#ax1.set_title('Test Accuracy ')

plt.xlabel('')

plt.savefig('linear SVM vs gauss rbf test acc for Emily.jpg', dpi= 300, bbox_inches='tight')

In [215]:
from __future__ import division
clfl2.decision_function(Xtrain1_7).shape
# Percent Agreement
sum(clfl2.predict(Xtrain1_7) == Ytrain1_7)/len(Ytrain1_7)*100.0000 #100%
sum(clfl2.predict(Xval1_7) == Yval1_7)/len(Yval1_7)*100.0000 #99%

99.0

In [236]:
lamda = 1e3
clfl2 = sk.linear_model.LogisticRegression(penalty = 'l2', tol = 1e-10, C = 1/lamda, 
                                              fit_intercept = True, random_state = 10, 
                                         max_iter = 1000, intercept_scaling = 1e6)
clfl2.fit(Xtrain1_7, Ytrain1_7)
(clfl2.predict(Xval1_7) == Yval1_7)[129] # False

False

In [ ]:
# Plot image of misclassified digit..makes sense
index = 129
fig = plt.figure(figsize=(5,5), dpi=300)

a =np.array(Xval1[index]).reshape((28, 28))
plt.imshow(a, cmap='hot', interpolation='nearest')
fig.savefig('l2 lambda = 1000 misclassif 1.jpg', bbox_inches='tight')

In [247]:
# corr classif
(clfl2.predict(Xval1_7) == Yval1_7)[1] # true
index = 1
fig = plt.figure(figsize=(5,5), dpi=300)

a =np.array(Xval1[index]).reshape((28, 28))
plt.imshow(a, cmap='hot', interpolation='nearest')
fig.savefig('l2 lambda = 1000 corr classif 1.jpg', bbox_inches='tight')

In [248]:
# corr classif
(clfl2.predict(Xval1_7) == Yval1_7)[151] # true
index = 151-150
fig = plt.figure(figsize=(5,5), dpi=300)

a =np.array(Xval7[index]).reshape((28, 28))
plt.imshow(a, cmap='hot', interpolation='nearest')
fig.savefig('l2 lambda = 1000 corr classif 7.jpg', bbox_inches='tight')

In [243]:
(clfl2.predict(Xval1_7) == Yval1_7)[227] # False
# Plot image of misclassified digit..makes sense
index = 227-150
fig = plt.figure(figsize=(5,5), dpi=300)

a =np.array(Xval7[index]).reshape((28, 28))
plt.imshow(a, cmap='hot', interpolation='nearest')
fig.savefig('l2 lambda = 1000 misclassif 7.jpg', bbox_inches='tight')

In [264]:
lamda = 1e3
clfl2 = sk.linear_model.LogisticRegression(penalty = 'l2', tol = 1e-10, C = 1/lamda, 
                                              fit_intercept = True, random_state = 10, 
                                         max_iter = 1000, intercept_scaling = 1e6)
clfl2.fit(Xtrain3_5, Ytrain3_5)
(clfl2.predict(Xval3_5) == Yval3_5)[235] # False

False

In [ ]:
# Plot image of misclassified digits..makes sense - for Emily
index = 30
fig = plt.figure(figsize=(5,5), dpi=300)
a =np.array(Xval3[index]).reshape((28, 28))
plt.imshow(a, cmap='hot', interpolation='nearest')
fig.savefig('l2 lambda = 1000 misclassif 3.jpg', bbox_inches='tight')

index = 1
fig = plt.figure(figsize=(5,5), dpi=300)
a =np.array(Xval3[index]).reshape((28, 28))
plt.imshow(a, cmap='hot', interpolation='nearest')
fig.savefig('l2 lambda = 1000 corr classif 3.jpg', bbox_inches='tight')

In [266]:
# Plot image of misclassified digits..makes sense - for Emily
index = 235-150
fig = plt.figure(figsize=(5,5), dpi=300)
a =np.array(Xval5[index]).reshape((28, 28))
plt.imshow(a, cmap='hot', interpolation='nearest')
fig.savefig('l2 lambda = 1000 misclassif 5.jpg', bbox_inches='tight')

index = 237-150
fig = plt.figure(figsize=(5,5), dpi=300)
a =np.array(Xval5[index]).reshape((28, 28))
plt.imshow(a, cmap='hot', interpolation='nearest')
fig.savefig('l2 lambda = 1000 corr classif 5.jpg', bbox_inches='tight')

In [217]:
_inferno_data = [[0.001462, 0.000466, 0.013866],
                 [0.002267, 0.001270, 0.018570],
                 [0.003299, 0.002249, 0.024239],
                 [0.004547, 0.003392, 0.030909],
                 [0.006006, 0.004692, 0.038558],
                 [0.007676, 0.006136, 0.046836],
                 [0.009561, 0.007713, 0.055143],
                 [0.011663, 0.009417, 0.063460],
                 [0.013995, 0.011225, 0.071862],
                 [0.016561, 0.013136, 0.080282],
                 [0.019373, 0.015133, 0.088767],
                 [0.022447, 0.017199, 0.097327],
                 [0.025793, 0.019331, 0.105930],
                 [0.029432, 0.021503, 0.114621],
                 [0.033385, 0.023702, 0.123397],
                 [0.037668, 0.025921, 0.132232],
                 [0.042253, 0.028139, 0.141141],
                 [0.046915, 0.030324, 0.150164],
                 [0.051644, 0.032474, 0.159254],
                 [0.056449, 0.034569, 0.168414],
                 [0.061340, 0.036590, 0.177642],
                 [0.066331, 0.038504, 0.186962],
                 [0.071429, 0.040294, 0.196354],
                 [0.076637, 0.041905, 0.205799],
                 [0.081962, 0.043328, 0.215289],
                 [0.087411, 0.044556, 0.224813],
                 [0.092990, 0.045583, 0.234358],
                 [0.098702, 0.046402, 0.243904],
                 [0.104551, 0.047008, 0.253430],
                 [0.110536, 0.047399, 0.262912],
                 [0.116656, 0.047574, 0.272321],
                 [0.122908, 0.047536, 0.281624],
                 [0.129285, 0.047293, 0.290788],
                 [0.135778, 0.046856, 0.299776],
                 [0.142378, 0.046242, 0.308553],
                 [0.149073, 0.045468, 0.317085],
                 [0.155850, 0.044559, 0.325338],
                 [0.162689, 0.043554, 0.333277],
                 [0.169575, 0.042489, 0.340874],
                 [0.176493, 0.041402, 0.348111],
                 [0.183429, 0.040329, 0.354971],
                 [0.190367, 0.039309, 0.361447],
                 [0.197297, 0.038400, 0.367535],
                 [0.204209, 0.037632, 0.373238],
                 [0.211095, 0.037030, 0.378563],
                 [0.217949, 0.036615, 0.383522],
                 [0.224763, 0.036405, 0.388129],
                 [0.231538, 0.036405, 0.392400],
                 [0.238273, 0.036621, 0.396353],
                 [0.244967, 0.037055, 0.400007],
                 [0.251620, 0.037705, 0.403378],
                 [0.258234, 0.038571, 0.406485],
                 [0.264810, 0.039647, 0.409345],
                 [0.271347, 0.040922, 0.411976],
                 [0.277850, 0.042353, 0.414392],
                 [0.284321, 0.043933, 0.416608],
                 [0.290763, 0.045644, 0.418637],
                 [0.297178, 0.047470, 0.420491],
                 [0.303568, 0.049396, 0.422182],
                 [0.309935, 0.051407, 0.423721],
                 [0.316282, 0.053490, 0.425116],
                 [0.322610, 0.055634, 0.426377],
                 [0.328921, 0.057827, 0.427511],
                 [0.335217, 0.060060, 0.428524],
                 [0.341500, 0.062325, 0.429425],
                 [0.347771, 0.064616, 0.430217],
                 [0.354032, 0.066925, 0.430906],
                 [0.360284, 0.069247, 0.431497],
                 [0.366529, 0.071579, 0.431994],
                 [0.372768, 0.073915, 0.432400],
                 [0.379001, 0.076253, 0.432719],
                 [0.385228, 0.078591, 0.432955],
                 [0.391453, 0.080927, 0.433109],
                 [0.397674, 0.083257, 0.433183],
                 [0.403894, 0.085580, 0.433179],
                 [0.410113, 0.087896, 0.433098],
                 [0.416331, 0.090203, 0.432943],
                 [0.422549, 0.092501, 0.432714],
                 [0.428768, 0.094790, 0.432412],
                 [0.434987, 0.097069, 0.432039],
                 [0.441207, 0.099338, 0.431594],
                 [0.447428, 0.101597, 0.431080],
                 [0.453651, 0.103848, 0.430498],
                 [0.459875, 0.106089, 0.429846],
                 [0.466100, 0.108322, 0.429125],
                 [0.472328, 0.110547, 0.428334],
                 [0.478558, 0.112764, 0.427475],
                 [0.484789, 0.114974, 0.426548],
                 [0.491022, 0.117179, 0.425552],
                 [0.497257, 0.119379, 0.424488],
                 [0.503493, 0.121575, 0.423356],
                 [0.509730, 0.123769, 0.422156],
                 [0.515967, 0.125960, 0.420887],
                 [0.522206, 0.128150, 0.419549],
                 [0.528444, 0.130341, 0.418142],
                 [0.534683, 0.132534, 0.416667],
                 [0.540920, 0.134729, 0.415123],
                 [0.547157, 0.136929, 0.413511],
                 [0.553392, 0.139134, 0.411829],
                 [0.559624, 0.141346, 0.410078],
                 [0.565854, 0.143567, 0.408258],
                 [0.572081, 0.145797, 0.406369],
                 [0.578304, 0.148039, 0.404411],
                 [0.584521, 0.150294, 0.402385],
                 [0.590734, 0.152563, 0.400290],
                 [0.596940, 0.154848, 0.398125],
                 [0.603139, 0.157151, 0.395891],
                 [0.609330, 0.159474, 0.393589],
                 [0.615513, 0.161817, 0.391219],
                 [0.621685, 0.164184, 0.388781],
                 [0.627847, 0.166575, 0.386276],
                 [0.633998, 0.168992, 0.383704],
                 [0.640135, 0.171438, 0.381065],
                 [0.646260, 0.173914, 0.378359],
                 [0.652369, 0.176421, 0.375586],
                 [0.658463, 0.178962, 0.372748],
                 [0.664540, 0.181539, 0.369846],
                 [0.670599, 0.184153, 0.366879],
                 [0.676638, 0.186807, 0.363849],
                 [0.682656, 0.189501, 0.360757],
                 [0.688653, 0.192239, 0.357603],
                 [0.694627, 0.195021, 0.354388],
                 [0.700576, 0.197851, 0.351113],
                 [0.706500, 0.200728, 0.347777],
                 [0.712396, 0.203656, 0.344383],
                 [0.718264, 0.206636, 0.340931],
                 [0.724103, 0.209670, 0.337424],
                 [0.729909, 0.212759, 0.333861],
                 [0.735683, 0.215906, 0.330245],
                 [0.741423, 0.219112, 0.326576],
                 [0.747127, 0.222378, 0.322856],
                 [0.752794, 0.225706, 0.319085],
                 [0.758422, 0.229097, 0.315266],
                 [0.764010, 0.232554, 0.311399],
                 [0.769556, 0.236077, 0.307485],
                 [0.775059, 0.239667, 0.303526],
                 [0.780517, 0.243327, 0.299523],
                 [0.785929, 0.247056, 0.295477],
                 [0.791293, 0.250856, 0.291390],
                 [0.796607, 0.254728, 0.287264],
                 [0.801871, 0.258674, 0.283099],
                 [0.807082, 0.262692, 0.278898],
                 [0.812239, 0.266786, 0.274661],
                 [0.817341, 0.270954, 0.270390],
                 [0.822386, 0.275197, 0.266085],
                 [0.827372, 0.279517, 0.261750],
                 [0.832299, 0.283913, 0.257383],
                 [0.837165, 0.288385, 0.252988],
                 [0.841969, 0.292933, 0.248564],
                 [0.846709, 0.297559, 0.244113],
                 [0.851384, 0.302260, 0.239636],
                 [0.855992, 0.307038, 0.235133],
                 [0.860533, 0.311892, 0.230606],
                 [0.865006, 0.316822, 0.226055],
                 [0.869409, 0.321827, 0.221482],
                 [0.873741, 0.326906, 0.216886],
                 [0.878001, 0.332060, 0.212268],
                 [0.882188, 0.337287, 0.207628],
                 [0.886302, 0.342586, 0.202968],
                 [0.890341, 0.347957, 0.198286],
                 [0.894305, 0.353399, 0.193584],
                 [0.898192, 0.358911, 0.188860],
                 [0.902003, 0.364492, 0.184116],
                 [0.905735, 0.370140, 0.179350],
                 [0.909390, 0.375856, 0.174563],
                 [0.912966, 0.381636, 0.169755],
                 [0.916462, 0.387481, 0.164924],
                 [0.919879, 0.393389, 0.160070],
                 [0.923215, 0.399359, 0.155193],
                 [0.926470, 0.405389, 0.150292],
                 [0.929644, 0.411479, 0.145367],
                 [0.932737, 0.417627, 0.140417],
                 [0.935747, 0.423831, 0.135440],
                 [0.938675, 0.430091, 0.130438],
                 [0.941521, 0.436405, 0.125409],
                 [0.944285, 0.442772, 0.120354],
                 [0.946965, 0.449191, 0.115272],
                 [0.949562, 0.455660, 0.110164],
                 [0.952075, 0.462178, 0.105031],
                 [0.954506, 0.468744, 0.099874],
                 [0.956852, 0.475356, 0.094695],
                 [0.959114, 0.482014, 0.089499],
                 [0.961293, 0.488716, 0.084289],
                 [0.963387, 0.495462, 0.079073],
                 [0.965397, 0.502249, 0.073859],
                 [0.967322, 0.509078, 0.068659],
                 [0.969163, 0.515946, 0.063488],
                 [0.970919, 0.522853, 0.058367],
                 [0.972590, 0.529798, 0.053324],
                 [0.974176, 0.536780, 0.048392],
                 [0.975677, 0.543798, 0.043618],
                 [0.977092, 0.550850, 0.039050],
                 [0.978422, 0.557937, 0.034931],
                 [0.979666, 0.565057, 0.031409],
                 [0.980824, 0.572209, 0.028508],
                 [0.981895, 0.579392, 0.026250],
                 [0.982881, 0.586606, 0.024661],
                 [0.983779, 0.593849, 0.023770],
                 [0.984591, 0.601122, 0.023606],
                 [0.985315, 0.608422, 0.024202],
                 [0.985952, 0.615750, 0.025592],
                 [0.986502, 0.623105, 0.027814],
                 [0.986964, 0.630485, 0.030908],
                 [0.987337, 0.637890, 0.034916],
                 [0.987622, 0.645320, 0.039886],
                 [0.987819, 0.652773, 0.045581],
                 [0.987926, 0.660250, 0.051750],
                 [0.987945, 0.667748, 0.058329],
                 [0.987874, 0.675267, 0.065257],
                 [0.987714, 0.682807, 0.072489],
                 [0.987464, 0.690366, 0.079990],
                 [0.987124, 0.697944, 0.087731],
                 [0.986694, 0.705540, 0.095694],
                 [0.986175, 0.713153, 0.103863],
                 [0.985566, 0.720782, 0.112229],
                 [0.984865, 0.728427, 0.120785],
                 [0.984075, 0.736087, 0.129527],
                 [0.983196, 0.743758, 0.138453],
                 [0.982228, 0.751442, 0.147565],
                 [0.981173, 0.759135, 0.156863],
                 [0.980032, 0.766837, 0.166353],
                 [0.978806, 0.774545, 0.176037],
                 [0.977497, 0.782258, 0.185923],
                 [0.976108, 0.789974, 0.196018],
                 [0.974638, 0.797692, 0.206332],
                 [0.973088, 0.805409, 0.216877],
                 [0.971468, 0.813122, 0.227658],
                 [0.969783, 0.820825, 0.238686],
                 [0.968041, 0.828515, 0.249972],
                 [0.966243, 0.836191, 0.261534],
                 [0.964394, 0.843848, 0.273391],
                 [0.962517, 0.851476, 0.285546],
                 [0.960626, 0.859069, 0.298010],
                 [0.958720, 0.866624, 0.310820],
                 [0.956834, 0.874129, 0.323974],
                 [0.954997, 0.881569, 0.337475],
                 [0.953215, 0.888942, 0.351369],
                 [0.951546, 0.896226, 0.365627],
                 [0.950018, 0.903409, 0.380271],
                 [0.948683, 0.910473, 0.395289],
                 [0.947594, 0.917399, 0.410665],
                 [0.946809, 0.924168, 0.426373],
                 [0.946392, 0.930761, 0.442367],
                 [0.946403, 0.937159, 0.458592],
                 [0.946903, 0.943348, 0.474970],
                 [0.947937, 0.949318, 0.491426],
                 [0.949545, 0.955063, 0.507860],
                 [0.951740, 0.960587, 0.524203],
                 [0.954529, 0.965896, 0.540361],
                 [0.957896, 0.971003, 0.556275],
                 [0.961812, 0.975924, 0.571925],
                 [0.966249, 0.980678, 0.587206],
                 [0.971162, 0.985282, 0.602154],
                 [0.976511, 0.989753, 0.616760],
                 [0.982257, 0.994109, 0.631017],
                 [0.988362, 0.998364, 0.644924]]

from matplotlib.colors import ListedColormap

inferno = ListedColormap(_inferno_data, name='inferno')

In [230]:
_viridis_data = [[0.267004, 0.004874, 0.329415],
                 [0.268510, 0.009605, 0.335427],
                 [0.269944, 0.014625, 0.341379],
                 [0.271305, 0.019942, 0.347269],
                 [0.272594, 0.025563, 0.353093],
                 [0.273809, 0.031497, 0.358853],
                 [0.274952, 0.037752, 0.364543],
                 [0.276022, 0.044167, 0.370164],
                 [0.277018, 0.050344, 0.375715],
                 [0.277941, 0.056324, 0.381191],
                 [0.278791, 0.062145, 0.386592],
                 [0.279566, 0.067836, 0.391917],
                 [0.280267, 0.073417, 0.397163],
                 [0.280894, 0.078907, 0.402329],
                 [0.281446, 0.084320, 0.407414],
                 [0.281924, 0.089666, 0.412415],
                 [0.282327, 0.094955, 0.417331],
                 [0.282656, 0.100196, 0.422160],
                 [0.282910, 0.105393, 0.426902],
                 [0.283091, 0.110553, 0.431554],
                 [0.283197, 0.115680, 0.436115],
                 [0.283229, 0.120777, 0.440584],
                 [0.283187, 0.125848, 0.444960],
                 [0.283072, 0.130895, 0.449241],
                 [0.282884, 0.135920, 0.453427],
                 [0.282623, 0.140926, 0.457517],
                 [0.282290, 0.145912, 0.461510],
                 [0.281887, 0.150881, 0.465405],
                 [0.281412, 0.155834, 0.469201],
                 [0.280868, 0.160771, 0.472899],
                 [0.280255, 0.165693, 0.476498],
                 [0.279574, 0.170599, 0.479997],
                 [0.278826, 0.175490, 0.483397],
                 [0.278012, 0.180367, 0.486697],
                 [0.277134, 0.185228, 0.489898],
                 [0.276194, 0.190074, 0.493001],
                 [0.275191, 0.194905, 0.496005],
                 [0.274128, 0.199721, 0.498911],
                 [0.273006, 0.204520, 0.501721],
                 [0.271828, 0.209303, 0.504434],
                 [0.270595, 0.214069, 0.507052],
                 [0.269308, 0.218818, 0.509577],
                 [0.267968, 0.223549, 0.512008],
                 [0.266580, 0.228262, 0.514349],
                 [0.265145, 0.232956, 0.516599],
                 [0.263663, 0.237631, 0.518762],
                 [0.262138, 0.242286, 0.520837],
                 [0.260571, 0.246922, 0.522828],
                 [0.258965, 0.251537, 0.524736],
                 [0.257322, 0.256130, 0.526563],
                 [0.255645, 0.260703, 0.528312],
                 [0.253935, 0.265254, 0.529983],
                 [0.252194, 0.269783, 0.531579],
                 [0.250425, 0.274290, 0.533103],
                 [0.248629, 0.278775, 0.534556],
                 [0.246811, 0.283237, 0.535941],
                 [0.244972, 0.287675, 0.537260],
                 [0.243113, 0.292092, 0.538516],
                 [0.241237, 0.296485, 0.539709],
                 [0.239346, 0.300855, 0.540844],
                 [0.237441, 0.305202, 0.541921],
                 [0.235526, 0.309527, 0.542944],
                 [0.233603, 0.313828, 0.543914],
                 [0.231674, 0.318106, 0.544834],
                 [0.229739, 0.322361, 0.545706],
                 [0.227802, 0.326594, 0.546532],
                 [0.225863, 0.330805, 0.547314],
                 [0.223925, 0.334994, 0.548053],
                 [0.221989, 0.339161, 0.548752],
                 [0.220057, 0.343307, 0.549413],
                 [0.218130, 0.347432, 0.550038],
                 [0.216210, 0.351535, 0.550627],
                 [0.214298, 0.355619, 0.551184],
                 [0.212395, 0.359683, 0.551710],
                 [0.210503, 0.363727, 0.552206],
                 [0.208623, 0.367752, 0.552675],
                 [0.206756, 0.371758, 0.553117],
                 [0.204903, 0.375746, 0.553533],
                 [0.203063, 0.379716, 0.553925],
                 [0.201239, 0.383670, 0.554294],
                 [0.199430, 0.387607, 0.554642],
                 [0.197636, 0.391528, 0.554969],
                 [0.195860, 0.395433, 0.555276],
                 [0.194100, 0.399323, 0.555565],
                 [0.192357, 0.403199, 0.555836],
                 [0.190631, 0.407061, 0.556089],
                 [0.188923, 0.410910, 0.556326],
                 [0.187231, 0.414746, 0.556547],
                 [0.185556, 0.418570, 0.556753],
                 [0.183898, 0.422383, 0.556944],
                 [0.182256, 0.426184, 0.557120],
                 [0.180629, 0.429975, 0.557282],
                 [0.179019, 0.433756, 0.557430],
                 [0.177423, 0.437527, 0.557565],
                 [0.175841, 0.441290, 0.557685],
                 [0.174274, 0.445044, 0.557792],
                 [0.172719, 0.448791, 0.557885],
                 [0.171176, 0.452530, 0.557965],
                 [0.169646, 0.456262, 0.558030],
                 [0.168126, 0.459988, 0.558082],
                 [0.166617, 0.463708, 0.558119],
                 [0.165117, 0.467423, 0.558141],
                 [0.163625, 0.471133, 0.558148],
                 [0.162142, 0.474838, 0.558140],
                 [0.160665, 0.478540, 0.558115],
                 [0.159194, 0.482237, 0.558073],
                 [0.157729, 0.485932, 0.558013],
                 [0.156270, 0.489624, 0.557936],
                 [0.154815, 0.493313, 0.557840],
                 [0.153364, 0.497000, 0.557724],
                 [0.151918, 0.500685, 0.557587],
                 [0.150476, 0.504369, 0.557430],
                 [0.149039, 0.508051, 0.557250],
                 [0.147607, 0.511733, 0.557049],
                 [0.146180, 0.515413, 0.556823],
                 [0.144759, 0.519093, 0.556572],
                 [0.143343, 0.522773, 0.556295],
                 [0.141935, 0.526453, 0.555991],
                 [0.140536, 0.530132, 0.555659],
                 [0.139147, 0.533812, 0.555298],
                 [0.137770, 0.537492, 0.554906],
                 [0.136408, 0.541173, 0.554483],
                 [0.135066, 0.544853, 0.554029],
                 [0.133743, 0.548535, 0.553541],
                 [0.132444, 0.552216, 0.553018],
                 [0.131172, 0.555899, 0.552459],
                 [0.129933, 0.559582, 0.551864],
                 [0.128729, 0.563265, 0.551229],
                 [0.127568, 0.566949, 0.550556],
                 [0.126453, 0.570633, 0.549841],
                 [0.125394, 0.574318, 0.549086],
                 [0.124395, 0.578002, 0.548287],
                 [0.123463, 0.581687, 0.547445],
                 [0.122606, 0.585371, 0.546557],
                 [0.121831, 0.589055, 0.545623],
                 [0.121148, 0.592739, 0.544641],
                 [0.120565, 0.596422, 0.543611],
                 [0.120092, 0.600104, 0.542530],
                 [0.119738, 0.603785, 0.541400],
                 [0.119512, 0.607464, 0.540218],
                 [0.119423, 0.611141, 0.538982],
                 [0.119483, 0.614817, 0.537692],
                 [0.119699, 0.618490, 0.536347],
                 [0.120081, 0.622161, 0.534946],
                 [0.120638, 0.625828, 0.533488],
                 [0.121380, 0.629492, 0.531973],
                 [0.122312, 0.633153, 0.530398],
                 [0.123444, 0.636809, 0.528763],
                 [0.124780, 0.640461, 0.527068],
                 [0.126326, 0.644107, 0.525311],
                 [0.128087, 0.647749, 0.523491],
                 [0.130067, 0.651384, 0.521608],
                 [0.132268, 0.655014, 0.519661],
                 [0.134692, 0.658636, 0.517649],
                 [0.137339, 0.662252, 0.515571],
                 [0.140210, 0.665859, 0.513427],
                 [0.143303, 0.669459, 0.511215],
                 [0.146616, 0.673050, 0.508936],
                 [0.150148, 0.676631, 0.506589],
                 [0.153894, 0.680203, 0.504172],
                 [0.157851, 0.683765, 0.501686],
                 [0.162016, 0.687316, 0.499129],
                 [0.166383, 0.690856, 0.496502],
                 [0.170948, 0.694384, 0.493803],
                 [0.175707, 0.697900, 0.491033],
                 [0.180653, 0.701402, 0.488189],
                 [0.185783, 0.704891, 0.485273],
                 [0.191090, 0.708366, 0.482284],
                 [0.196571, 0.711827, 0.479221],
                 [0.202219, 0.715272, 0.476084],
                 [0.208030, 0.718701, 0.472873],
                 [0.214000, 0.722114, 0.469588],
                 [0.220124, 0.725509, 0.466226],
                 [0.226397, 0.728888, 0.462789],
                 [0.232815, 0.732247, 0.459277],
                 [0.239374, 0.735588, 0.455688],
                 [0.246070, 0.738910, 0.452024],
                 [0.252899, 0.742211, 0.448284],
                 [0.259857, 0.745492, 0.444467],
                 [0.266941, 0.748751, 0.440573],
                 [0.274149, 0.751988, 0.436601],
                 [0.281477, 0.755203, 0.432552],
                 [0.288921, 0.758394, 0.428426],
                 [0.296479, 0.761561, 0.424223],
                 [0.304148, 0.764704, 0.419943],
                 [0.311925, 0.767822, 0.415586],
                 [0.319809, 0.770914, 0.411152],
                 [0.327796, 0.773980, 0.406640],
                 [0.335885, 0.777018, 0.402049],
                 [0.344074, 0.780029, 0.397381],
                 [0.352360, 0.783011, 0.392636],
                 [0.360741, 0.785964, 0.387814],
                 [0.369214, 0.788888, 0.382914],
                 [0.377779, 0.791781, 0.377939],
                 [0.386433, 0.794644, 0.372886],
                 [0.395174, 0.797475, 0.367757],
                 [0.404001, 0.800275, 0.362552],
                 [0.412913, 0.803041, 0.357269],
                 [0.421908, 0.805774, 0.351910],
                 [0.430983, 0.808473, 0.346476],
                 [0.440137, 0.811138, 0.340967],
                 [0.449368, 0.813768, 0.335384],
                 [0.458674, 0.816363, 0.329727],
                 [0.468053, 0.818921, 0.323998],
                 [0.477504, 0.821444, 0.318195],
                 [0.487026, 0.823929, 0.312321],
                 [0.496615, 0.826376, 0.306377],
                 [0.506271, 0.828786, 0.300362],
                 [0.515992, 0.831158, 0.294279],
                 [0.525776, 0.833491, 0.288127],
                 [0.535621, 0.835785, 0.281908],
                 [0.545524, 0.838039, 0.275626],
                 [0.555484, 0.840254, 0.269281],
                 [0.565498, 0.842430, 0.262877],
                 [0.575563, 0.844566, 0.256415],
                 [0.585678, 0.846661, 0.249897],
                 [0.595839, 0.848717, 0.243329],
                 [0.606045, 0.850733, 0.236712],
                 [0.616293, 0.852709, 0.230052],
                 [0.626579, 0.854645, 0.223353],
                 [0.636902, 0.856542, 0.216620],
                 [0.647257, 0.858400, 0.209861],
                 [0.657642, 0.860219, 0.203082],
                 [0.668054, 0.861999, 0.196293],
                 [0.678489, 0.863742, 0.189503],
                 [0.688944, 0.865448, 0.182725],
                 [0.699415, 0.867117, 0.175971],
                 [0.709898, 0.868751, 0.169257],
                 [0.720391, 0.870350, 0.162603],
                 [0.730889, 0.871916, 0.156029],
                 [0.741388, 0.873449, 0.149561],
                 [0.751884, 0.874951, 0.143228],
                 [0.762373, 0.876424, 0.137064],
                 [0.772852, 0.877868, 0.131109],
                 [0.783315, 0.879285, 0.125405],
                 [0.793760, 0.880678, 0.120005],
                 [0.804182, 0.882046, 0.114965],
                 [0.814576, 0.883393, 0.110347],
                 [0.824940, 0.884720, 0.106217],
                 [0.835270, 0.886029, 0.102646],
                 [0.845561, 0.887322, 0.099702],
                 [0.855810, 0.888601, 0.097452],
                 [0.866013, 0.889868, 0.095953],
                 [0.876168, 0.891125, 0.095250],
                 [0.886271, 0.892374, 0.095374],
                 [0.896320, 0.893616, 0.096335],
                 [0.906311, 0.894855, 0.098125],
                 [0.916242, 0.896091, 0.100717],
                 [0.926106, 0.897330, 0.104071],
                 [0.935904, 0.898570, 0.108131],
                 [0.945636, 0.899815, 0.112838],
                 [0.955300, 0.901065, 0.118128],
                 [0.964894, 0.902323, 0.123941],
                 [0.974417, 0.903590, 0.130215],
                 [0.983868, 0.904867, 0.136897],
                 [0.993248, 0.906157, 0.143936]]

from matplotlib.colors import ListedColormap

viridis = ListedColormap(_viridis_data, name='viridis')